#Ingestion del archivo country.json


### Paso 1 - Leer el archivo JSON usando "DataframeReader"

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run 
"../includes/common_functions"


In [0]:
%run
"../includes/configuration"

In [0]:
country_schema = ("countryId int, countryIsoCode string, countryName string")



In [0]:
country_df = spark.read \
    .schema(country_schema)\
    .json(f"{bronze_folder_path}/{v_file_date}/country.json")
display(country_df)

In [0]:
display(country_df)

## Parte 2 : Eliminar columnas no deseadas del dataframe

In [0]:
from pyspark.sql.functions import col


In [0]:
country_dropped_df = country_df.drop("countryIsoCode")
# otra manera country_df.drop(col("countryIsoCode"))
display(country_dropped_df)


## Paso 3. Cambiar el nombre de las columnas segun lo requerido y añadir "ingestion date" y "enviroment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
country_final_df = country_dropped_df \
    .withColumnRenamed("countryId", "country_id") \
    .withColumnRenamed("countryName", "country_name") \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("enviroment", lit("Production")) \
    .withColumn("file_date", lit(v_file_date))




    

### Paso 4. Salida en un formato parquet

In [0]:
country_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.countries")

In [0]:
%sql
select * from movie_silver.countries


In [0]:
dbutils.notebook.exit("Ejecutado exitosamente")